# clinical ner transformer

In this notebook I use pretrained clinic_bioBert encoder transformer to extract diagnosis and medications (NER and classification) from medical notes data.

### Main Objectives

1. Generate or Load Data

- Option A: Synthetic EMRs 
- Option B: Open datasets: BC5CDR (Diseases/Chemicals): https://huggingface.co/datasets/biocreative_cdr

2. BIO Tag the Data

- Label scheme: B-ENTITY, I-ENTITY, O(no entity).

3. Tokenize

- Use the same tokenizer as the model (handles subwords + offset mapping).

4. Load Pretrained Model

- Bio_ClinicalBERT: emilyalsentzer/Bio_ClinicalBERT

5. Fine-Tune

- Objective: add token classification head (NER).
- Loss: cross-entropy over token labels (ignore specials with -100).

6. Classify Diagnoses & Meds

- Postprocess logits - entity spans.
- Aggregate subwords back to words; merge B-/I- runs.

7. Evaluate

- Metrics: precision / recall / F1 (seqeval).
- Inspect errors (boundary splits, abbreviations, synonyms).

### install and import necessary modules

In [ ]:
!conda install -y pytorch cpuonly -c pytorch

In [ ]:
!pip install -U transformers seqeval evaluate bioc
!pip install "datasets<4.0.0" fsspec pyarrow

In [1]:
import transformers
from datasets import load_dataset
import torch

In [2]:
print("transformers:", transformers.__version__)
print("torch:", torch.__version__)

transformers: 4.56.0
torch: 2.6.0


### load dataset from hugging face datasets

The BioCreative V Chemical Disease Relation (CDR) dataset is a large annotated text corpus of human annotations of all chemicals, diseases and their interactions in 1,500 PubMed articles.

In [3]:
dataset = load_dataset("bigbio/bc5cdr", "bc5cdr_source", trust_remote_code=True)    

In [69]:
# dataset content
dataset

DatasetDict({
    train: Dataset({
        features: ['passages'],
        num_rows: 500
    })
    test: Dataset({
        features: ['passages'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['passages'],
        num_rows: 500
    })
})

### observe dataset features and structure - 


Each row in train / test / validation contains one document.
Inside it, the passages field is a list of two items:

- Title (type = "title")

- Abstract (type = "abstract")

Each passage has:

- text: the actual string

- entities: list of labeled entities (with offsets + text + type + normalized DB links)

- relations: relations between entities (e.g. chemical–disease links)

In [70]:
passage_feature = dataset["train"].features['passages']
passage_feature

[{'document_id': Value(dtype='string', id=None),
  'type': Value(dtype='string', id=None),
  'text': Value(dtype='string', id=None),
  'entities': [{'id': Value(dtype='string', id=None),
    'offsets': [[Value(dtype='int32', id=None)]],
    'text': [Value(dtype='string', id=None)],
    'type': Value(dtype='string', id=None),
    'normalized': [{'db_name': Value(dtype='string', id=None),
      'db_id': Value(dtype='string', id=None)}]}],
  'relations': [{'id': Value(dtype='string', id=None),
    'type': Value(dtype='string', id=None),
    'arg1_id': Value(dtype='string', id=None),
    'arg2_id': Value(dtype='string', id=None)}]}]

random example from dataset 

In [71]:
dataset["train"][200] 

{'passages': [{'document_id': '6747681',
   'type': 'title',
   'text': 'Intra-arterial BCNU chemotherapy for treatment of malignant gliomas of the central nervous system.',
   'entities': [{'id': '0',
     'offsets': [[15, 19]],
     'text': ['BCNU'],
     'type': 'Chemical',
     'normalized': [{'db_name': 'MESH', 'db_id': 'D002330'}]},
    {'id': '1',
     'offsets': [[50, 67]],
     'text': ['malignant gliomas'],
     'type': 'Disease',
     'normalized': [{'db_name': 'MESH', 'db_id': 'D005910'}]}],
   'relations': [{'id': 'R0',
     'type': 'CID',
     'arg1_id': 'D002330',
     'arg2_id': 'D031300'}]},
  {'document_id': '6747681',
   'type': 'abstract',
   'text': 'Because of the rapid systemic clearance of BCNU (1,3-bis-(2-chloroethyl)-1-nitrosourea), intra-arterial administration should provide a substantial advantage over intravenous administration for the treatment of malignant gliomas. Thirty-six patients were treated with BCNU every 6 to 8 weeks, either by transfemoral cath

### Build BIO tags dataset

For each entity offset, mark the corresponding tokens with B-(TYPE) and I-(TYPE).
This converts the BigBio dataset into a standard token classification dataset.

In [4]:
import re
from typing import List, Dict

label_lst = ["O","B-Chemical","I-Chemical","B-Disease","I-Disease"]
label_to_id = {l:i for i,l in enumerate(label_lst)}

def separate_to_words(text: str):
    # get text and return a separated words list with their start and end indexes
    # output shape: List[Tuple[str,int,int]]
    return [(m.group(0), m.start(), m.end()) for m in re.finditer(r"[A-Za-z]+(?:'[A-Za-z]+)?|[^\w\s]", text)]

def get_entities(passage: Dict):
    # get features dictionary and return an entity type list with their start and end indexes
    # output shape: List[Tuple[str,int,int]]
    spans = []
    for ent in passage.get("entities", []):
        # BigBio offsets: [[start, end]], extract min and max
        offs = ent.get("offsets", [])
        if not offs: 
            continue
        s = min(o[0] for o in offs)
        e = max(o[1] for o in offs)
        t = ent.get("type", "")
        spans.append((s, e, t))
    return spans

def merge_passages(passages: List[Dict], joiner=" "):
    # get a dict that contains two passages and merge them. return the full text and its entities
    # output shape: text, List[Tuple[int,int,str]]
    full, ents, offset = [], [], 0
    for p in passages:
        text = p.get("text","")
        full.append(text)
        full.append(joiner)  # add white space between title and abstract passages
        for (s,e,t) in get_entities(p):
            ents.append((s+offset, e+offset, t))
        offset += len(text) + len(joiner)
    if full and full[-1] == joiner:
        full.pop()  # drop redundant whitespace
 
    return "".join(full), ents

def main_doc_to_words_tags(doc: Dict) -> Dict[str, List]:
    text, entity_spans = merge_passages(doc["passages"])
    toks = separate_to_words(text)
    words  = [w for (w,_,_) in toks]
    tags   = ["O"] * len(words)

    # map BigBio types to two classes
    def map_type(t: str):
        t = t.lower()
        if t.startswith("chem"): 
            return "Chemical"
        if t.startswith("dis"):  
            return "Disease"
        return None

    # assign BIO by overlap and continuity
    for i, (tok, s, e) in enumerate(toks):
        # find overlapping entity with max overlap
        best = None; best_ov = 0
        for (es, ee, t) in entity_spans:
            ov = min(e, ee) - max(s, es)
            if ov > best_ov:
                best_ov = ov
                best = (es, ee, t)
        if best and best_ov > 0:
            es, ee, t = best
            cls = map_type(t)
            if cls:
                # B- if token starts inside entity at or before entity start; else I-
                tags[i] = "B-"+cls if s <= es < e else "I-"+cls

    # enforce I- continuity
    for i in range(len(tags)):
        if tags[i].startswith("I-"):
            if i == 0 or tags[i-1] == "O" or tags[i-1][2:] != tags[i][2:]:
                tags[i] = "B-" + tags[i][2:]

    ner_ids = [label_to_id.get(t, 0) for t in tags]
    return {"words": words, "ner_tags": ner_ids}

In [5]:
from datasets import Dataset, DatasetDict, Features, Sequence, Value, ClassLabel

processed = {}
for split in dataset.keys():  # train test and val
    rows = [main_doc_to_words_tags(doc) for doc in dataset[split]]
    ds_split = Dataset.from_list(rows)
    feats = Features({
        "words": Sequence(Value("string")),
        "ner_tags": Sequence(ClassLabel(names=label_lst))
    })
    processed[split] = ds_split.cast(feats)

token_level_ds = DatasetDict(processed)
print(token_level_ds)

Casting the dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['words', 'ner_tags'],
        num_rows: 500
    })
    test: Dataset({
        features: ['words', 'ner_tags'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['words', 'ner_tags'],
        num_rows: 500
    })
})


present some examples of post bio tag dataset

In [76]:
words = token_level_ds["train"][200]["words"][:20]
labels = token_level_ds["train"][200]["ner_tags"][:20]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_lst[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Intra - arterial BCNU       chemotherapy for treatment of malignant gliomas   of the central nervous system . Because of the rapid 
O     O O        B-Chemical O            O   O         O  B-Disease I-Disease O  O   O       O       O      O O       O  O   O     


### Apply Hugging Face tokenizer

In [6]:
from transformers import AutoTokenizer

# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [7]:
def tokenize_and_align_labels(example):
    # tokenize words
    tokenized = tokenizer(example["words"], is_split_into_words=True, truncation=True)
    word_ids = tokenized.word_ids()

    aligned_labels = []
    previous_word_id = None
    for word_id in word_ids:
        if word_id is None:
            # special tokens (CLS, SEP, PAD)
            aligned_labels.append(-100)
        elif word_id != previous_word_id:
            # first subword → take label
            aligned_labels.append(example["ner_tags"][word_id])
        else:
            # Same word as previous token
            label = example["ner_tags"][word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            aligned_labels.append(label)
        previous_word_id = word_id

    tokenized["labels"] = aligned_labels
    return tokenized

In [8]:
tokenized_datasets = token_level_ds.map(
    tokenize_and_align_labels,
    batched=False,
    remove_columns=token_level_ds["train"].column_names
)
print(tokenized_datasets["train"][0].keys())

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [9]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_datasets["train"][200]["input_ids"])
labels = tokenized_datasets["train"][200]["labels"]

for t, l in zip(tokens[:30], labels[:30]):
    label_name = label_lst[l] if l != -100 else "IGN"
    print(f"{t:15} {label_name}")


[CLS]           IGN
in              O
##tra           O
-               O
art             O
##erial         O
b               B-Chemical
##c             I-Chemical
##nu            I-Chemical
ch              O
##em            O
##otherapy      O
for             O
treatment       O
of              O
ma              B-Disease
##li            I-Disease
##gnant         I-Disease
g               I-Disease
##lio           I-Disease
##mas           I-Disease
of              O
the             O
central         O
nervous         O
system          O
.               O
because         O
of              O
the             O


### Fine-tuning the model

#### define a data collector

pads input and labels of each batch to be the same length

In [9]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

#### define metrics using seqeval

In [10]:
import numpy as np
import evaluate

metric = evaluate.load("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_lst[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_lst[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

#### load pretrained model


In [11]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

label_to_id = {l:i for i,l in enumerate(label_lst)}
id_to_label = {i:l for i,l in enumerate(label_lst)}

num_labels = len(label_lst)

model = AutoModelForTokenClassification.from_pretrained(
    "emilyalsentzer/Bio_ClinicalBERT",
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model.config.num_labels

5

#### define TrainingArguments and the trainer

In [21]:
!pip install -U "accelerate>=0.26.0"


In [12]:
training_args = TrainingArguments(
    output_dir="ner_clinicalbert_bc5cdr",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=42,
    fp16=torch.cuda.is_available()  # mixed precision if GPU
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [15]:
# custom training loop

num_epochs = 3
training_batch = 8
for epoch in num_epochs:
    

#### Fine-tune & Evaluate

In [ ]:
trainer.train()


In [ ]:
# Validation metrics (best checkpoint already loaded if load_best_model_at_end=True)
val_metrics = trainer.evaluate()
print("Validation:", val_metrics)

# Test set evaluation
test_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
print("Test:", test_metrics)

#### Save the fine-tuned model (for FastAPI, batch jobs, etc.)

In [ ]:
save_dir = "ner_clinicalbert_bc5cdr_best"
trainer.model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print("Saved to:", save_dir)